In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas
import datetime
import os
import sys
module_path = os.path.abspath(os.path.join('../data_pipeline/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import sqlite3
import create_onset_data

In [2]:
inds_dbdir = "/home/bharat/Documents/data/ss_onset_dataset/data/sqlite3/"
sm_db_name = "smu_sml_sme.sqlite"
sm_table_name = "smusmlsme"

In [ ]:
dataObj = create_onset_data.OnsetData(delTCutoff=2)
ssBinDF = dataObj.create_output_bins(\
                saveFile="../data/binned_data_extra_latlon.feather",\
                aulDBdir=inds_dbdir, \
                 aulDBName=sm_db_name,\
                 aulTabName=sm_table_name)
# print ssBinDF.head()
# print "-------------"
# print ssBinDF.shape

In [ ]:
ssBinDF.tail(6000)["data_label"].value_counts()